El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## 1. Preparación de datos

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from math import sqrt
import time
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import re


In [2]:
# %pip install -U --user scikit-learn
# %pip install -U --user lightgbm
# %pip install -U --user catboost
# %pip install -U --user xgboost


In [3]:
# df = pd.read_csv('/datasets/car_data.csv')

Importacion del dataframe

In [4]:
df = pd.read_csv('D:/Tripleten/datasets/car_data.csv')

Visualización general del dataframe

In [5]:
df.sample(4)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
339028,26/03/2016 18:06,9000,small,2012,manual,101,corsa,60000,12,petrol,opel,no,26/03/2016 00:00,0,60437,06/04/2016 18:19
302404,30/03/2016 22:54,9000,NaN,2017,NaN,0,transporter,5000,7,gasoline,volkswagen,NaN,30/03/2016 00:00,0,42107,05/04/2016 19:15
233579,28/03/2016 22:36,18000,coupe,1985,auto,125,other,150000,7,gasoline,mercedes_benz,no,28/03/2016 00:00,0,76571,05/04/2016 12:45
233104,01/04/2016 21:46,2780,convertible,1996,manual,90,golf,150000,3,lpg,volkswagen,no,01/04/2016 00:00,0,49716,05/04/2016 19:44


El objetivo del proyecto se basará en la predicción del precio por lo que prepararemos los datos para un  modelo de regresión.


Se renombran las columnas para mantener formato snake_case en el código.

In [6]:
cols = df.columns
snake_case_cols = []

for col in cols:
    snake_case_col = '_'.join(re.findall(r'[A-Z][a-z0-9]*', col)).lower()
    snake_case_cols.append(snake_case_col)

# print(snake_case_cols)
df.columns = snake_case_cols

Obteniendo información general

In [7]:
df.sample(5)

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
215721,18/03/2016 12:47,1399,NaN,2018,manual,90,golf,150000,1,NaN,volkswagen,NaN,18/03/2016 00:00,0,95460,29/03/2016 12:49
49175,12/03/2016 07:58,5900,sedan,2007,manual,105,golf,150000,9,gasoline,volkswagen,no,12/03/2016 00:00,0,92318,12/03/2016 09:43
325158,19/03/2016 23:52,4800,wagon,2003,auto,170,c_klasse,150000,7,gasoline,mercedes_benz,no,19/03/2016 00:00,0,22049,20/03/2016 07:40
126513,30/03/2016 23:52,16500,sedan,2005,auto,334,a8,150000,6,petrol,audi,no,30/03/2016 00:00,0,84028,06/04/2016 00:17
215077,16/03/2016 09:51,3600,wagon,2006,manual,0,focus,150000,6,gasoline,ford,no,16/03/2016 00:00,0,44339,06/04/2016 02:44


Revisando la estructura del dataframe

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   mileage             354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

Identificando columnas con valores nulos

In [9]:
null_values = df.isna().sum()
null_rows= df.shape[0] - df.dropna().shape[0]

null_values = null_values[null_values>0]


print(f'Null values by column:')
print(null_values, end='\n\n')
print(f'Total null rows: {null_rows}')

Null values by column:
vehicle_type    37490
gearbox         19833
model           19705
fuel_type       32895
not_repaired    71154
dtype: int64

Total null rows: 108555


Identificacndo valores duplicados

In [10]:
duplicated_rows = df.duplicated()
duplicated_rows.sum()

262

Examinando manualmente un caso para verificar coherencia

In [11]:
df[(df['date_crawled'] == '21/03/2016 19:06') & (df['price']== 5999)]
# df.query("date_crawled == '21/03/2016 19:06' and price == 5999")

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
183,21/03/2016 19:06,5999,small,2009,manual,80,polo,125000,5,petrol,volkswagen,no,21/03/2016 00:00,0,65529,05/04/2016 20:47
14266,21/03/2016 19:06,5999,small,2009,manual,80,polo,125000,5,petrol,volkswagen,no,21/03/2016 00:00,0,65529,05/04/2016 20:47


Observaciones:

- Elementos con clasificaciones Dtype erroneos (price, not_repaired, date_crawled, last_seen)
- Elementos nulos para las columnas (vehicle_type, gearbox, model, fuel_type, not_repaired)
- 262 elementos duplicados.

## 2. Limpieza de datos

Modificamos la columna 'not_repaired' a booleano

In [12]:
# pd.set_option('future.no_silent_downcasting', True)
df['not_repaired']= df['not_repaired'].replace({'yes':1, 'no':0})

C:\Users\jm_he\AppData\Local\Temp\ipykernel_5844\2022677092.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['not_repaired']= df['not_repaired'].replace({'yes':1, 'no':0})


Se reasignaran los tipos de valor 'date_crawled' y 'last_seen' al formato 'datetime64'

In [13]:
df['date_crawled'] = pd.to_datetime(df['date_crawled'], format='%d/%m/%Y %H:%M')
df['last_seen'] = pd.to_datetime(df['last_seen'], format='%d/%m/%Y %H:%M')

Verificando cambios realizados

In [14]:
print(df['date_crawled'].dt.year.unique() == df['last_seen'].dt.year.unique())
# print(df['date_crawled'].dt.month.sort_values().unique() == df['last_seen'].dt.month.sort_values().unique())

[ True]


Procedemos a eliminar los valores duplicados

In [15]:
df = df.drop_duplicates().reset_index(drop=True)

Se crea una función que permita analizar visualmente los elementos únicos que se encuentran dentro de las columnas (para verificar datos incongruentes)



In [16]:
def analyze(db):
    obj = db.select_dtypes(include='object').columns
    int = db.select_dtypes(include='int').columns
    
    print('Analisis de elementos Object')
    for i in obj:

        print(f'Elementos de la columna {i}')
        print(db[i].sort_values().unique(), end='\n\n')

    for i in int:

        print(f'Elementos de la columna {i}')
        print(db[i].sort_values().unique(), end='\n\n')



In [17]:
analyze(df)

Analisis de elementos Object
Elementos de la columna vehicle_type
['bus' 'convertible' 'coupe' 'other' 'sedan' 'small' 'suv' 'wagon' nan]

Elementos de la columna gearbox
['auto' 'manual' nan]

Elementos de la columna model
['100' '145' '147' '156' '159' '1_reihe' '1er' '200' '2_reihe' '300c'
 '3_reihe' '3er' '4_reihe' '500' '5_reihe' '5er' '601' '6_reihe' '6er'
 '7er' '80' '850' '90' '900' '9000' '911' 'a1' 'a2' 'a3' 'a4' 'a5' 'a6'
 'a8' 'a_klasse' 'accord' 'agila' 'alhambra' 'almera' 'altea' 'amarok'
 'antara' 'arosa' 'astra' 'auris' 'avensis' 'aveo' 'aygo' 'b_klasse'
 'b_max' 'beetle' 'berlingo' 'bora' 'boxster' 'bravo' 'c1' 'c2' 'c3' 'c4'
 'c5' 'c_klasse' 'c_max' 'c_reihe' 'caddy' 'calibra' 'captiva' 'carisma'
 'carnival' 'cayenne' 'cc' 'ceed' 'charade' 'cherokee' 'citigo' 'civic'
 'cl' 'clio' 'clk' 'clubman' 'colt' 'combo' 'cooper' 'cordoba' 'corolla'
 'corsa' 'cr_reihe' 'croma' 'crossfire' 'cuore' 'cx_reihe' 'defender'
 'delta' 'discovery' 'doblo' 'ducato' 'duster' 'e_klasse' 'el

In [18]:
def chart_prev(df):
    fig, axes =plt.subplots(ncols=4,nrows=4,figsize=[12,12])

    for ax, column in zip(axes.flatten(), df.columns):
        ax.hist(df[column].dropna(), bins=10, color='g')
        ax.grid(True)
        ax.set_title(column)

    plt.tight_layout()
    plt.show()
    

In [19]:
# chart_prev(df)


### Observaciones


Al momento nos encontramos con las siguientes situaciones:

Elementos nulos para las columnas:
- vehicle_type
- gearbox 
- model
- fuel_type
- not_repaired

A su vez, hemos identificado incongruencias en los datos registrados para las siguientes columnas: 

- `price`: En los valores de precio puedo observar valores de muy bajos que pueden ser considerados `sesgos de información.`
- `registration year` : se pueden observar ver valores superiores al año actual y valores por debajo de 1886 (cuando se creó el primer vehículo), existen muchos `datos con sesgo`.
- `power` la medición esta en caballos de vapor. El primer vehiculo creado tenía una fuerza de CV de 0.75, por lo que es incongruente ver valores de 0 en la base de datos, en su contraparte, el vehículo con mayor velocidad de CV regitrada para 2020 fue de 1800 CV, por lo que `tenemos incongruencias` en los datos cuyos valores superan los a 10,000 CV. 

De un total de 16 columnas, 8 columnas se encuentran sesgadas

#### Sesgo de 'price'

`- Existen precios de vehículos por debajo de los 600 dolares.`

Para poder ejecutar un análisis mas profundo es necesario identificar que pudo haber pasado con los datos, una suposición sería que la base se descargo de una plataforma en línea donde los usuarios pueden decidir el precio final de su vehículo, algunos usuarios podrían optar por colocar precios irreales para llamar la atención de los compradores.

Con el fin de alimentar nuestro modelo de una mejor manera, será necesario reclasificar o excluir los precios menores a 600 dolares. Este umbral se ha obtenido de la busqueda de los precios minimos y maximos para la venta de vehiculos en europa de los sitios
- www.coches.com,
- www.milanuncios.com

Si realizaramos un filtro de precios encontrariamos 46,313 filas afectadas que representan el 13.07% del total del dataset

#### Sesgo de 'registration_year'
`- Se pueden observar ver valores superiores al año actual y valores por debajo de 1886 (antes del primer automovil creado).`

Siguiendo las fuentes antes mencionadas, podemos ver vehículos ofertados con fecha de registro desde 1980, mantendremos este umbral en nuestro dataset, por otra parte el registro muestra vehiculos con fecha de registro mayor a 2016, fecha en la que se descargaron estos datos. Por lo que datos posteriores no deberían de considerarse para este estudio.

Considerando la depuración del umbral mínimo y máximo, sería necesario depurar 17,978 filas representando un 5.07% del total del dataset.

#### Sesgo de 'power'
`- Valores incongruentes en los CV, con valores inferiores a 60 y superiores a 500 en nuestra base de datos.`

Para la seccion de power tenemos como entrada que el primer vehiculo construido tenía una potencia de Caballos de Vapor (CV) de 0.75. 
De acuerdo con las paginas antes mencionadas, los vehiculos pueden tener una potencia entre 60 y 400 cv


Se obtuvieron muestras al azar los vehiculos de la base de datos con registro mayor a 400cv y ningúno clasifico con mas de 250 cv reales, por lo que limitaremos la búsqueda a 300 cv
Si manejamos los umbrales de 60,300 tendríamos un total de 67,268  datos por de purar representando un 18.99% del dataset.


#### Sesgo de 'mileage'
`- Existen vehiculos con registro de salida del año en curso y con 150,000 km registrados (datos incongruentes).`

En promedio, los vehiculos generarn un kilometraje de 15,000 a 27,000 kms por año, para minimizar los datos afectados, se excluiran los autos con un kilometraje mayor a 150,000 para añós posteriores a 2011. (27 kms por añó).
19102 filas que corresponden a un 5.39 del dataset


### Observaciones

El dataset descargado presenta bastantes datos sesgados e incongruentes, si bien hemos establecido la hipótesis de que la base se descargo de una plataforma en línea donde los usuarios pueden decidir el precio final de su vehículo no podemos asegurar que sea correcta, sin embargo el enfoque principal del de este proyecto se basa en entrenar un modelo de predicción, por lo que será necesario excluir estos datos para tener una base de mejor calidad.

In [20]:
# Excluyendo valores price
recalibrate_df= df.copy()

In [21]:
recalibrate_df= recalibrate_df.query('price>=600')
recalibrate_df = recalibrate_df.query(' 1980 < registration_year < 2017')
recalibrate_df = recalibrate_df.query('60 <=power <=300')
recalibrate_df = recalibrate_df.query('~(registration_year>=2012 and mileage >= 150000)')

In [22]:
loss_data = (1 - (recalibrate_df.shape[0] / df.shape[0])) *100
print(f'Se ha depurado un {loss_data:.2f} de la data original')

Se ha depurado un 30.42 de la data original


In [23]:
recalibrate_df.isna().sum().sort_values(ascending=False)

not_repaired          30254
fuel_type              9408
model                  7475
vehicle_type           3932
gearbox                3357
date_crawled              0
price                     0
registration_year         0
power                     0
mileage                   0
registration_month        0
brand                     0
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
dtype: int64

In [24]:
# chart_prev(recalibrate_df)

Con estos cambios las columnas de 'power' y de 'registration_year' mejoraron su coherencia. El kilometraje sigue estando alto para autos con valores de recorrido mayor a 150,000, sin embargo, de acuerdo con la información externa, esta puede entrar en el umbral de 27,000 kms por año.

### Valores nuloss

Para la manipulación de los valores nulos, se han analizado las siguientes opciones:
`- Eliminación de filas o columnas con valores nulos`
- Imputación de valores nulos
- Uso de modelos de imputación
- Codificación de valores nulos

Inicialmente se optó por modelos de imputación sin embargo el peso de la base de datos no permitio generar las predicciones necesarias, por lo que se optará por la eliminación de filas con valores nulos.

In [25]:
imputated_df = recalibrate_df.dropna()


Mantenemos las columnas que realmente nos interesan

In [26]:
imputated_df = imputated_df.drop(columns=['date_crawled','date_created','number_of_pictures', 'last_seen','postal_code'])

# chart_prev(imputated_df)


In [27]:
imputated_df.sample(5)

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired
261853,6990,wagon,2008,manual,170,c5,150000,7,gasoline,citroen,0.0
322611,11300,wagon,2010,manual,102,a3,100000,11,petrol,audi,0.0
156697,11900,small,2013,manual,120,2_reihe,40000,1,petrol,peugeot,0.0
227133,3333,small,2008,manual,76,twingo,100000,5,petrol,renault,0.0
59100,2500,sedan,2003,manual,116,bora,150000,3,petrol,volkswagen,0.0


## 3. Transformación de los datos

Definimos la funciones necesarias para la tranformación de datos.

In [28]:
def OHE(df):
    '''Esta funcion recibira un dataframe con columnas categóricas y devolverá su version codificada en multiples columnas binarias'''
    encoder = OneHotEncoder(drop='first')
    encoded_data = encoder.fit_transform(df)
    output_names = encoder.get_feature_names_out(df.columns)
    df_codified = pd.DataFrame(encoded_data.toarray(), columns=output_names, index= df.index)
    return df_codified

In [29]:
def scaler(X_train, X_test):
    '''Esta función procesara las caracteristicas de entrenamiento y devolverá un escalamiento de los datos numéricos'''
    int_names = X_train.select_dtypes(include=['int']).columns
    model =  StandardScaler()
    X_train[int_names] = model.fit_transform(X_train[int_names])
    X_test[int_names] = model.transform(X_test[int_names])
    return X_train, X_test

Aplicamos OHE a los valores categórigos.

In [30]:
obj_cols = imputated_df.select_dtypes(include=['object']).columns
new_cols = OHE(imputated_df[obj_cols])

imputated_df =imputated_df.drop(columns=obj_cols)
imputated_df = pd.concat([imputated_df, new_cols], axis=1)

Definimos nuestras características y nuestros objectivos para el proyecto.

In [31]:
X = imputated_df.drop(columns=['price'])
y = imputated_df['price']

Verificando si los elementos fueron procesados correctamente

In [32]:
X.head()

,registration_year,power,mileage,registration_month,not_repaired,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,...,brand_rover,brand_saab,brand_seat,brand_skoda,brand_smart,brand_subaru,brand_suzuki,brand_toyota,brand_volkswagen,brand_volvo
3,2001,75,150000,6,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2008,69,90000,7,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1995,102,150000,10,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2004,109,150000,8,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2004,105,150000,12,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
y.head()

3     1500
4     3600
5      650
6     2200
10    2000
Name: price, dtype: int64

## 4. Entrenamiento del modelo 

Se entrenarán los modelos 'Linear_Regression', 'Random_Forest', 'LightGBM', 'Catboost' y 'XGboost' para comparar los mejores resultados del RMSE. Los primeros 3 elementos usaran la función model_training. Para los modelos 'Catboost' 'XGboost' los entrenamientos se realizarán con código independiente.

In [34]:
def model_training(X,y, model, params):
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=100) 
    X_train, X_test = scaler(X_train, X_test)

    grid_search = GridSearchCV(model, params, scoring='neg_root_mean_squared_error', cv=5, verbose=3)
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)

    rmse = sqrt(mean_squared_error(y_test,y_pred))

    return rmse

La siguiente variable 'times', nos permitirá conocer el tiempo de ejecución de cada modelo

In [35]:
times = []

### Linear Regression Model 

In [36]:
start_time = time.time()

model = LinearRegression()
params = {
    'fit_intercept': [True, False],
}
rmse_lr = model_training(X,y,model,params)

end_time = time.time()
elapsed_time = end_time - start_time


times.append({'Modelo':'Linear Regression','RMSE':rmse_lr,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (Linear Regression):", rmse_lr)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ............fit_intercept=True;, score=-2296.642 total time=  10.8s
[CV 2/5] END ............fit_intercept=True;, score=-2308.902 total time=  10.9s
[CV 3/5] END ............fit_intercept=True;, score=-2297.597 total time=  10.7s
[CV 4/5] END ............fit_intercept=True;, score=-2286.359 total time=  10.2s
[CV 5/5] END ............fit_intercept=True;, score=-2312.306 total time=  10.0s
[CV 1/5] END ...........fit_intercept=False;, score=-2300.274 total time=  11.3s
[CV 2/5] END ...........fit_intercept=False;, score=-2312.251 total time=   7.2s
[CV 3/5] END ...........fit_intercept=False;, score=-2300.640 total time=   4.8s
[CV 4/5] END ...........fit_intercept=False;, score=-2289.403 total time=   4.4s
[CV 5/5] END ...........fit_intercept=False;, score=-2315.262 total time=   5.0s


Tiempo de ejecución: 93.02 segundos
RMSE en el conjunto de entrenamiento (Linear Regression): 2305.9333785873855


### Bosque Aleatorio (Regresión)

In [37]:
start_time = time.time()

model = RandomForestRegressor(random_state=100)
params = {
    'n_estimators': [10],
    # 'n_estimators': [10,20,100],
    # 'max_depth': [None, 10,20]
    # 'min_samples_split': [2,10],
    # 'min_samples_leaf': [1,4]
}
rmse_rf = model_training(X,y,model,params)

end_time = time.time()
elapsed_time = end_time - start_time

times.append({'Modelo':'Random_Forest','RMSE':rmse_rf,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (Random Forest):", rmse_rf)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ...............n_estimators=10;, score=-1598.319 total time=  21.4s
[CV 2/5] END ...............n_estimators=10;, score=-1598.641 total time=  21.0s
[CV 3/5] END ...............n_estimators=10;, score=-1596.968 total time=  20.8s
[CV 4/5] END ...............n_estimators=10;, score=-1608.618 total time=  20.5s
[CV 5/5] END ...............n_estimators=10;, score=-1614.533 total time=  20.7s


Tiempo de ejecución: 132.95 segundos
RMSE en el conjunto de entrenamiento (Random Forest): 1576.3102511586826


### LightGBM Regressor

In [38]:
start_time = time.time()

y= imputated_df['price']
model = LGBMRegressor(objective='regression', metric='rmse', random_state=100)
params = {

    'boosting_type': ['gbdt','dart'],
    # 'num_leaves': [20,30,40],
    'learning_rate': [0.05,0.1,0.2],
    'n_estimators': [50,100]

    # 'boosting_type': ['gbdt','dart'],
    # 'num_leaves': [20,30,40],
    # 'learning_rate': [0.05,0.1,0.2],
    # 'n_estimators': [50,100,200]
}
rmse_lgbm = model_training(X,y,model,params)

end_time = time.time()
elapsed_time = end_time - start_time

times.append({'Modelo':'Light_GBM','RMSE':rmse_lgbm,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (Lightgbm):", rmse_lgbm)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\jm_he\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\jm_he\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1008.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004117 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 820
[LightGBM] [Info] Number of data points in the train set: 122072, number of used features: 271
[LightGBM] [Info] Start training from score 5690.157899
[CV 1/5] END boosting_type=gbdt, learning_rate=0.05, n_estimators=50;, score=-1914.959 total time=   2.2s
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003587 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 821
[LightGBM] [Info] Number of data points in the train set: 122072, number of used features: 271
[LightGBM] [Info] Start training from score 5687.642719
[CV 2/5] END boosting_type=gbdt, learning_rate=0.05, n_estimators=50;, sco

### Catboost Regressor

In [39]:
start_time = time.time()

# Inicializar el modelo de CatBoost
catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=6, loss_function='RMSE', verbose=0)

# Entrenar el modelo de CatBoost
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=100) 
X_train, X_test = scaler(X_train, X_test)

catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Predecir sobre el conjunto de entrenamiento y prueba
y_train_pred_catboost = catboost_model.predict(X_train)
y_test_pred_catboost = catboost_model.predict(X_test)

# Calcular el error cuadrático medio (RMSE) en el conjunto de entrenamiento y prueba
rmse_train_catboost = sqrt(mean_squared_error(y_train, y_train_pred_catboost))
rmse_test_catboost = sqrt(mean_squared_error(y_test, y_test_pred_catboost))

end_time = time.time()
elapsed_time = end_time - start_time

times.append({'Modelo':'Catboost','RMSE':rmse_test_catboost,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (CatBoost):", rmse_train_catboost)
print("RMSE en el conjunto de prueba (CatBoost):", rmse_test_catboost)




Tiempo de ejecución: 18.28 segundos
RMSE en el conjunto de entrenamiento (CatBoost): 1527.0330315524154
RMSE en el conjunto de prueba (CatBoost): 1563.529273376928


### XGboost

In [40]:
start_time = time.time()

# Convertir los conjuntos de datos a formato DMatrix de XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Definir los parámetros del modelo
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Entrenar el modelo de XGBoost
xgb_model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dtrain, 'train'), (dtest, 'test')], early_stopping_rounds=100, verbose_eval=100)

# Predecir sobre el conjunto de entrenamiento y prueba
y_train_pred_xgb = xgb_model.predict(dtrain)
y_test_pred_xgb = xgb_model.predict(dtest)

# Calcular el error cuadrático medio (RMSE) en el conjunto de entrenamiento y prueba
rmse_train_xgb = sqrt(mean_squared_error(y_train, y_train_pred_xgb))
rmse_test_xgb = sqrt(mean_squared_error(y_test, y_test_pred_xgb))

end_time = time.time()
elapsed_time = end_time - start_time

times.append({'Modelo':'XGBoost','RMSE':rmse_test_xgb,'Tiempo':elapsed_time})

print('\n')
print(f'Tiempo de ejecución: {elapsed_time:.2f} segundos')
print("RMSE en el conjunto de entrenamiento (XGBoost):", rmse_train_xgb)
print("RMSE en el conjunto de prueba (XGBoost):", rmse_test_xgb)


[0]	train-rmse:4307.30326	test-rmse:4307.70386
[100]	train-rmse:1588.82314	test-rmse:1630.02950
[200]	train-rmse:1518.22112	test-rmse:1577.58946
[300]	train-rmse:1474.17489	test-rmse:1548.39210
[400]	train-rmse:1439.96363	test-rmse:1527.82914
[500]	train-rmse:1411.60735	test-rmse:1513.81693
[600]	train-rmse:1387.14323	test-rmse:1501.99304
[700]	train-rmse:1366.31467	test-rmse:1492.90538
[800]	train-rmse:1348.29044	test-rmse:1486.57075
[900]	train-rmse:1332.72549	test-rmse:1481.61230
[999]	train-rmse:1316.90864	test-rmse:1476.86578


Tiempo de ejecución: 37.11 segundos
RMSE en el conjunto de entrenamiento (XGBoost): 1316.9086390799503
RMSE en el conjunto de prueba (XGBoost): 1476.8657840883823


## 5. Análisis del modelo

In [41]:
results = pd.DataFrame(times).sort_values(by='RMSE').reset_index(drop=True)
results

,Modelo,RMSE,Tiempo
0,XGBoost,1476.865784,37.108805
1,Catboost,1563.529273,18.281880
2,Light_GBM,1564.172769,163.987820
3,Random_Forest,1576.310251,132.953465
4,Linear Regression,2305.933379,93.019858


### Conclusiones

Durante la preparación de los datos, se identificaron 5 columnas con valores nulos y otras 3 con datos incongruentes, dada la robustes de la base de datos se concluyó excluir las filas de los valores afectados.

La codificación de valores categóricos a través de OHE y la escala de los valores numéricos fueron aplicados durante el desarrollo de los modelos.

De acuerdo a los resultados mostrados en el dataframe, XGBoost es el modelo con mejor RMSE con un umbral de predicción de 1476 euros, seguido de Catboost y Light_GBM. En conclusión los modelos de potenciación de gradiente tuvieron mejores desempeños que aquellos que no lo aplican. 

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores
- [x]  Las celdas con el código han sido colocadas en orden de ejecución
- [x]  Los datos han sido descargados y preparados
- [x] Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos